In [3]:
from glob import glob
import re
import json
def loadFile(file_name) :
    with open(file_name, "r") as f :
        return "".join(f.readlines())
    return ""

def saveFile(file_name, contents) :
    with open(file_name, "w") as f :
        f.write(contents)

def loadJson(json_file_path) :
    with open(json_file_path, encoding='utf-8') as json_file :
        return json.load(json_file)

simpleApi = loadJson("docs_api.json")["children"]



In [15]:
def getClassInfo(className : str) :
    for info in simpleApi :
        if (info["name"] == className) :
            return info
    return None

def getSourceFile(info) :
    if "sources" in info :
        fileName : str = info["sources"][0]["fileName"]
        if fileName.startswith("src/app/three/") :
            return fileName[14:-3]
    return "error"

def isDecorators(info, decoType) :
    isInput = False
    if 'decorators' in info :
        decorators = info['decorators']
        for deco in decorators :
            if deco['name'] == decoType :
                isInput = True
    return isInput

def getExtends(info) :
    extendeds = []
    if "extendedTypes" in info : 
        extendedTypeName = info["extendedTypes"][0]["name"]
        childInfo = getClassInfo(extendedTypeName)
        if childInfo is not None :
            parentExtends = getExtends(childInfo)
            if len(parentExtends) > 0 :
                for name in parentExtends :
                    extendeds.append(name)
        extendeds.append(extendedTypeName)
    return extendeds

def getComment(info) :
    if "comment" not in info :
        return ""
    comment = info['comment']
    commentHtml = ''
    if 'shortText' in comment and 'text' in comment :
        commentHtml = '{0}<br/><br/>\n\t\t\t{1}'.format(
                comment['shortText'],
                "<br/>\n\t\t\t".join(comment['text'].replace("<br>","\n")
                    .replace("<br/>","\n")
                    .replace("<br />","\n")
                    .strip()
                    .replace("\n\n\n","\n\n")
                    .split("\n")
                )
        )
    elif 'shortText' in comment :
        commentHtml = '{0}'.format(
                comment['shortText']
            )
    if 'tags' in comment :
        tagHtml = []
        for tags in comment['tags'] :
            if (tags['tag'] == 'see'):
                tagText = tags['text'].strip()
                if tagText.endswith(",") : 
                    tagText = tagText[0:-1]
                threeSelector = re.compile("THREE\.([a-zA-Z0-9]+)")
                matches = threeSelector.findall(tagText)
                if matches :
                    tagHtml.append("[page:{0} {1}]".format(
                        matches[0],
                        tagText
                    ))
                else :
                    tagHtml.append(tagText) 
               
        if len(tagHtml) > 0 :
            commentHtml = '{0}\n\n\t\t\t<br/><br/>\n\t\t\t{1}'.format(
                commentHtml,
                "<br/>\n\t\t\t".join(tagHtml).strip()
            )
    if 'returns' in comment :
        commentHtml = '{0}\n\n\t\t\t<br/><br/>\n\t\t\tReturns : {1}'.format(
            commentHtml,
            comment['returns'].strip()
        )
    return commentHtml

def getParamTypes(child) :
    typeName = ''
    if 'type' in child :
        childType = child['type']['type']
        if childType == 'union' :
            typeNames = []
            for typeInfo in child['type']['types'] :
                if 'name' in typeInfo :
                    typeNames.append(typeInfo['name'])
                elif typeInfo['type'] == 'array':
                    typeNames.append(typeInfo['elementType']['name'])
            typeName = '|'.join(typeNames)
        elif childType == 'array' : 
            if 'elementType' in child['type'] and 'name' in child['type']['elementType'] :
                typeName = child['type']['elementType']['name'] + "[]"
            #else  :
                #print("Error")
                #print(child['type'])
        elif 'name' in child['type']:
            typeName = child['type']['name']
    return typeName

def getGenApiDocs(className : str) :
    info = getClassInfo(className)
    if info is None :
        return False
    kindString = info["kindString"]
    html = []
    html.append('<!DOCTYPE html>')
    html.append('<html lang="en">')
    html.append('	<head>')
    html.append('		<meta charset="utf-8" />')
    html.append('		<base href="../../" />')
    html.append('		<script src="page.js"></script>')
    html.append('		<link type="text/css" rel="stylesheet" href="page.css" />')
    html.append('	</head>')
    html.append('	<body>')
    if kindString == "Type alias" :
        typeName = info['type']
        print("--------------")
        print(getParamTypes(info))
        #types = info['type']['types']
    elif kindString == "Class" :
        if "children" in info:
            extends = getExtends(info)
            if len(extends) > 0 :
                extendHtml = []
                for ext in extends :
                    extendHtml.append("[page:{0}] &rarr;".format(ext))
                html.append('		' + " ".join(extendHtml))
                html.append('')
            html.append('		<h1>[name]</h1>')
            html.append('')
            html.append('		<p class="desc">\n\t\t\t{0}\n\t\t</p>'.format(getComment(info)))
            html.append('')
            if isDecorators(info, 'Component') :
                decorators = info['decorators']
                for deco in decorators :
                    name = deco['name']
                    if name == 'Component' :
                        selector = re.compile("selector: '([a-z0-9\-]+)'")
                        matches = selector.findall(deco['arguments']['obj'])
                        if matches :
                            tagName = matches[0]
                            html.append('		<h2>{0}</h2>'.format(deco['name']))
                            html.append('		<p>')
                            html.append('			&lt;{0}&gt;&lt;/{0}&gt;'.format(tagName))
                            html.append('		</p>')
                            html.append('')
            fileName = getSourceFile(info)

            html.append('		<h2>Properties - Input</h2>')
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common input properties.'.format(ext))
                    html.append('		</p>')
                html.append('')

            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                if sources == fileName and kindString == 'Property' and isDecorators(child, 'Input') :
                    typeName = getParamTypes(child)
                    html.append('		<h3>[property:{1} {0}]</h3>'.format(
                        child['name'], 
                        typeName
                    ))
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(getComment(child)))
                    html.append('')

            html.append('		<h2>Properties - Output</h2>')
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common output properties.'.format(ext))
                    html.append('		</p>')
                html.append('')

            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                if sources == fileName and kindString == 'Property' and isDecorators(child, 'Output') :
                    typeName = getParamTypes(child)
                    html.append('		<h3>[property:{1} {0}]</h3>'.format(
                        child['name'], 
                        typeName
                    ))
                    html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(getComment(child)))
                    html.append('')

            html.append('		<h2>Methods</h2>')
            if len(extends) > 0 :
                for ext in extends :
                    html.append('		<p>')
                    html.append('			See the base [page:{0} {0}] class for common methods.'.format(ext))
                    html.append('		</p>')
                html.append('')
            for child in info['children'] :        
                sources = getSourceFile(child)
                kindString = child['kindString']
                flags = child['flags']
                if sources == fileName and kindString == 'Method' and (
                    ('isPublic' in flags and flags['isPublic']) or ('isProtected' in flags and flags['isProtected'])) :
                    params = []
                    returnType  = 'void' 
                    comment = None
                    if 'signatures' in child :
                        for signatures in child['signatures'] :
                            if 'type' in signatures and 'name' in signatures['type'] :
                                returnType = signatures['type']['name']
                            comment = getComment(signatures)
                            if 'parameters' in signatures :
                                for parameters in signatures['parameters'] :
                                    typeName = getParamTypes(parameters)
                                    params.append(" [param:{0} {1}]".format(typeName, parameters['name'])); 
                    html.append('		<h3>[method:{1} {0}]( {2} )</h3>'.format(
                        child['name'], 
                        returnType,
                        ", ".join(params)
                    ))
                    if comment is not None :
                        html.append('		<p>\n\t\t\t{0}\n\t\t</p>'.format(comment))
                    html.append('')
    if 'sources' in info :
        sourcesPath = info['sources'][0]['fileName']
        html.append('		<h2>Source</h2>')
        html.append('		<p>[link:https://github.com/outmindkjg/three-sample/tree/master/src/app/three/{0}.ts {0}.ts]</p>'.format(sourcesPath[14:-3]))
    html.append('	</body>')
    html.append('</html>')
    saveFile('src/assets/ngxapi/en/{0}.html'.format(className),"\n".join(html))
    return True


In [16]:
# getGenApiDocs("MeshComponent")
# getGenApiDocs("RendererComponent")
getGenApiDocs("GeometryComponent")


True

In [17]:
loadedClass = {}
jsonList = {}
for child in simpleApi :
    name = child["name"]
    if name not in loadedClass:
        loadedClass[name] = child['sources'][0]['fileName']

for name in loadedClass :
    url = loadedClass[name]
    if url.startswith('src/app/three/') :
        if getGenApiDocs(name) :
            jsonList[name] = "ngxapi/en/{0}".format(name)    
print(jsonList)

--------------
number|Int8Array|Int16Array|Int32Array|Uint8Array|Uint16Array|Uint32Array|Float32Array|Float64Array|THREE.BufferAttribute
--------------
MeshMaterialRaw|THREE.Scene
--------------
string|number|THREE.Color
--------------
string|THREE.Texture|TextureOption|any
--------------
THREE.IUniform
--------------

{'AbstractGeometryComponent': 'ngxapi/en/AbstractGeometryComponent', 'AbstractMaterialComponent': 'ngxapi/en/AbstractMaterialComponent', 'AbstractObject3dComponent': 'ngxapi/en/AbstractObject3dComponent', 'AbstractTextureComponent': 'ngxapi/en/AbstractTextureComponent', 'BaseComponent': 'ngxapi/en/BaseComponent', 'CameraComponent': 'ngxapi/en/CameraComponent', 'GeometryComponent': 'ngxapi/en/GeometryComponent', 'MaterialComponent': 'ngxapi/en/MaterialComponent', 'MeshComponent': 'ngxapi/en/MeshComponent', 'RendererComponent': 'ngxapi/en/RendererComponent', 'SceneComponent': 'ngxapi/en/SceneComponent', 'TextureComponent': 'ngxapi/en/TextureComponent', 'ThreeClock': 'ngxap